### Import the library

In [1]:
import torch
import pickle
import random
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch_geometric.nn as gnn

from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split
from torch_geometric.data import Data

In [2]:
from net import PSOGNN
from pso import PSO 
from base_function import *

### Data Preparation

In [3]:
pickle_path = r'A:\Code\deepso\parameter_dataset.pkl'
with open(pickle_path, 'rb') as f:
    all_functions_by_dim = pickle.load(f)

In [4]:
print(all_functions_by_dim[1]['ackley'][0])
print(all_functions_by_dim[1]['ackley'])

[17.73616119740118, 0.15972786943049672, 7.811337914570187]
[[17.73616119740118, 0.15972786943049672, 7.811337914570187], [18.904159263252232, 0.19998814226956296, 5.124730900509964], [20.629512299056536, 0.219657518937181, 7.0563964919646], [16.45589323560797, 0.20161056926119264, 6.122304653244048], [22.783915131305253, 0.29463871392258256, 4.946596147216237], [18.659833923317965, 0.20117158459568685, 5.404887203290376], [18.949392509782026, 0.10712238373919303, 6.4384431578829435], [15.393661230018093, 0.26526223508142366, 6.89176676173339], [16.755771994939394, 0.2730069277629874, 5.085263781686141], [16.274870836911866, 0.13917554573333063, 4.738942679368025], [23.55946781668235, 0.2432630593711587, 6.081598793244506], [17.0708239416501, 0.24799733447894068, 5.588929413395962], [18.217231374537207, 0.221679496113465, 6.325156806274553], [22.35918827894029, 0.29462730899236383, 5.339487513301901], [16.265092240052596, 0.2228274321782594, 5.6147466166195645], [22.97486424525769, 0.2

In [5]:
train_data_combined = []
test_data_combined = []

for d, func_dict in all_functions_by_dim.items():
    for func_type, func_list in func_dict.items():
        if func_type == "largeman" or func_type == "crossintray" or func_type == "dropwave" :
            continue
        
        train_funcs, test_funcs = train_test_split(func_list, test_size=0.3, random_state=42)

        train_data_combined.extend([{'dim': d, 'func_type': func_type, 'params': params} for params in train_funcs])
        test_data_combined.extend([{'dim': d, 'func_type': func_type, 'params': params} for params in test_funcs])

print('split done')


split done


In [6]:
print(train_data_combined[1])

{'dim': 1, 'func_type': 'ackley', 'params': [22.07132405161137, 0.1138493715321388, 7.134177489177596]}


In [7]:
splited_data = {'train': train_data_combined, 'test': test_data_combined}
splited_path = r'A:\Code\deepso\splited_data_no.pkl'
with open(pickle_path, 'wb') as f:
    pass
with open(splited_path, 'wb') as f:
    pickle.dump(splited_data, f)

print(f"All functions split into combined train and test sets and saved to '{splited_path}'")

All functions split into combined train and test sets and saved to 'A:\Code\deepso\splited_data_no.pkl'


In [8]:
def create_batches(dataset, batch_size): 
    random.shuffle(dataset)
    for i in range(0, len(dataset), batch_size): 
        batch = dataset[i:i + batch_size]
        yield batch